In [1]:
import quandl
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
from statsmodels.tsa.stattools import adfuller
from sklearn import model_selection
from sklearn import linear_model
from sklearn.preprocessing import MinMaxScaler

In [2]:
def warn(*args, **kwargs):
    pass


warnings.warn = warn


def predictUs(thisTestX, mtl_thisTrain, llc_thisTrain, hr_thisTrain, mten_thisTrain, ridge_thisTrain, bayes_thisTrain):
    y_pred_mtl = mtl_thisTrain.predict(thisTestX).ravel()
    y_pred_llc = llc_thisTrain.predict(thisTestX).ravel()
    y_pred_hr = hr_thisTrain.predict(thisTestX).ravel()
    y_pred_mten = mten_thisTrain.predict(thisTestX).ravel()
    y_pred_ridge = ridge_thisTrain.predict(thisTestX).ravel()
    y_pred_bayes = bayes_thisTrain.predict(thisTestX).ravel()
    return y_pred_mtl, y_pred_llc, y_pred_hr, y_pred_mten, y_pred_ridge, y_pred_bayes

def separateTestingData8HoursAhead(thisData, window_size):
    panjang = len(thisData)
    panjang_window = window_size

    # Temporal data initialization
    tmp = np.transpose(thisData)

    # Training data initialization
    x_data = tmp[:, 0:panjang_window]
    y_data = tmp[:, panjang_window + 1:panjang_window + 2]

    # Filling the remaining training data
    for x in range(1, panjang - panjang_window - 1):
        tmp = np.roll(tmp, -1, axis=1)
        x_data = np.vstack((x_data, tmp[:, 0:panjang_window]))
        y_data = np.vstack(
            (y_data, tmp[:, panjang_window + 1:panjang_window + 2]))

    # Cleaning temporal data
    del tmp

    return x_data, y_data


def getTrainParams(thisTrainX, thisTrainY):
    mtl_thisTrain = linear_model.MultiTaskLasso()
    llc_thisTrain = linear_model.LassoLarsCV()
    hr_thisTrain = linear_model.HuberRegressor()
    mten_thisTrain = linear_model.MultiTaskElasticNetCV()
    ridge_thisTrain = linear_model.RidgeCV()
    bayes_thisTrain = linear_model.BayesianRidge()
    
    mtl_thisTrain.fit(thisTrainX, thisTrainY)
    llc_thisTrain.fit(thisTrainX, thisTrainY.ravel())
    hr_thisTrain.fit(thisTrainX, thisTrainY.ravel())
    mten_thisTrain.fit(thisTrainX, thisTrainY)
    ridge_thisTrain.fit(thisTrainX, thisTrainY)
    bayes_thisTrain.fit(thisTrainX, thisTrainY)

    return mtl_thisTrain, llc_thisTrain, hr_thisTrain, mten_thisTrain, ridge_thisTrain, bayes_thisTrain

def CalculateAccuracy(predicted_data, soon_data, current_data):
    deviationpredict = predicted_data - data_soon
    deviationreal = current_data - data_soon

    predictionmatrix = (deviationpredict > 0) * 1
    realmatrix = (deviationreal > 0) * 1

    result = (predictionmatrix == realmatrix) * 1
    print(sum(result) / len(result))

In [3]:
data = quandl.get("WGC/GOLD_DAILY_USD", authtoken="C5CQZuxZM7EbJvzMBGhm")
data.tail(5)

,Value
Date,
2018-11-05,1232.25
2018-11-06,1231.60
2018-11-07,1229.95
2018-11-08,1224.15
2018-11-09,1211.40


In [4]:
datalength = data.shape[0]//2
datatrain = data.iloc[:datalength,0:1].values
datatest = data.iloc[datalength:,0:1].values

In [5]:
window = 31
x_train_data, y_train_data = separateTestingData8HoursAhead(
    datatrain, window)

x_test_data, y_test_data = separateTestingData8HoursAhead(
    datatest, window)

In [6]:
mtl, llc, hr, mten, ridge, bayes = getTrainParams(x_train_data, y_train_data)

In [7]:
pred_mtl, pred_llc, pred_hr, pred_mten, pred_ridge, pred_bayes = predictUs(x_test_data, mtl, llc, hr, mten, ridge, bayes)

In [8]:
data_soon = data.iloc[datalength + window - 1: -2, 0:1].values.ravel()
current_data = data.iloc[datalength + window + 1:, 0:1].values.ravel()

In [9]:
#mtl
CalculateAccuracy(pred_mtl, data_soon, current_data)

0.48640832851359167


In [10]:
#llc
CalculateAccuracy(pred_llc, data_soon, current_data)

0.48544438018122227


In [11]:
#hr
CalculateAccuracy(pred_hr, data_soon, current_data)

0.49546944283786387


In [12]:
#mten
CalculateAccuracy(pred_mten, data_soon, current_data)

0.4918064391748602


In [13]:
#ridge
CalculateAccuracy(pred_ridge, data_soon, current_data)

0.48467322151532677


In [14]:
CalculateAccuracy(pred_bayes, data_soon, current_data)

0.4852515905147484
